In [1]:
import numpy as np
from electroviz.core.experiment import Experiment

exp = Experiment("E:/20221205_155624/")

stimuli = exp.stimuli
pop0 = exp.populations[0]
pop0 = pop0.remove(pop0.units["total_spikes"] < 1000)
pop0_sort = pop0.sort("depth")

C:\Ryan\GitHub\electroviz\electroviz\streams\spikes.py:105: RuntimeWarning: Mean of empty slice
  cluster_depths.append(np.nanmean(spike_depths[cluster_idx]))


In [2]:
from electroviz.core.kernel import SparseNoiseKernel
unit = pop0_sort[25]
stim = stimuli[0]
kern = SparseNoiseKernel(unit, stim)
kern.plot_norm_delay()

In [3]:
from electroviz.core.kernel import StaticGratingsKernel
unit = pop0_sort[25]
stim = stimuli[0]
kern = StaticGratingsKernel(unit, stim)
kern.plot_norm_delay()